In [ ]:
import cv2
import os
from keras.models import load_model
!pip install tqdm
from tqdm import tqdm
import json
import numpy as np

In [ ]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import load_model

# Ensure the haarcascade file is in the correct path
cascPath = os.path.abspath(os.path.dirname(os.path.dirname(os.getcwd()))) + "/haarcascade_frontalface_alt.xml"
# Check if the file exists
if not os.path.isfile(cascPath):
    raise FileNotFoundError(f"File not found: {cascPath}")
# Load the Haar Cascade classifier
faceCascade = cv2.CascadeClassifier(cascPath)
# Load the pre-trained emotion detection model
model = load_model("emotion_model.keras")

# Function to process video and save frames with predictions
def runGetImage(video_path, output_path):
    # capture video
    video_capture = cv2.VideoCapture(video_path)
    # get total frames
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # check if video opened successfully
    if not video_capture.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    # create empty list to store frames
    frames = []

    # loop through video frames
    for _ in tqdm(range(total_frames), desc="Processing frames"):
        # read frame from video
        ret, frame = video_capture.read()
        # check if frame is read correctly
        if not ret:
            break

        # convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect faces in the frame
        faces = faceCascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE
        )

        # check if any faces are detected
        if len(faces) > 0:
            # take the largest face detected
            x, y, w, h = max(faces, key=lambda face: face[2] * face[3])
            # draw rectangle around the face 
            face = gray[y:y+h, x:x+w]
            # resize the face to 48x48 pixels
            face_resized = cv2.resize(face, (48, 48))
            # normalize the pixel values to be between 0 and 1
            face_input = face_resized.reshape(1, 48, 48, 1).astype('float32') / 255.0

            # predict the emotion using the model
            result = model.predict(face_input, verbose=0)  # shape (1, N)
            # get the index of the class with the highest probability
            frames.append(result[0].tolist())
        else:
            # if no face is detected, append an empty list
            frames.append([])

    # release the video capture object
    video_capture.release()

    # write the frames to a JSON file
    # create the output directory if it doesn't exist
    with open(output_path, "w") as f:
        json.dump(frames, f, indent=2)

    print(f"Finished processing {video_path}. Saved output to {output_path}")

In [ ]:
# Path to the directory containing the videos
# and the output directory for the JSON files
video_paths = "<path_to_video_directory>"
output = "<path_to_output_directory>"

# Iterate through all video files in the directory
for v in os.listdir(video_paths):
    # Extract the emotion data from the video and save it to a JSON file
    runGetImage(os.path.join(video_paths,v), os.path.join(output, v[:-4] +".json"))